# Inferencia y Validación del Modelo de Sentimientos
**Propósito:** Este notebook simula el entorno de producción (API). Carga el pipeline entrenado y procesa entradas de texto individuales para devolver una respuesta en formato JSON.

### Estructura de Salida (Contrato de API)
Para mantener la compatibilidad con el frontend, el modelo devuelve:
- `text`: El texto original analizado.
- `prevision`: Etiqueta predicha (`Positivo`, `Negativo`, `Neutro`).
- `probabilidad`: Nivel de confianza del modelo (0.0 a 1.0).

In [2]:
import joblib
import numpy as np
import json

# Ruta al artefacto generado en el Notebook 02
MODEL_PATH = '../data/models/modelo_sentimiento_pipeline.pkl'

def load_inference_pipeline():
    """Carga el modelo y verifica su integridad."""
    try:
        model = joblib.load(MODEL_PATH)
        return model
    except Exception as e:
        print(f"Error al cargar el modelo: {e}")
        return None

pipeline = load_inference_pipeline()

def predict_to_json(input_text):
    """
    Transforma la entrada en el formato JSON requerido por la API.
    """
    if not input_text.strip():
        return {"error": "El texto está vacío"}

    # Predicción directa (El pipeline ya tiene el Vectorizador)
    label = pipeline.predict([input_text])[0]
    
    # LinearSVC no tiene predict_proba. 
    # Usamos decision_function + sigmoide para estimar una 'confianza' de 0 a 1.
    decision_score = pipeline.decision_function([input_text])
    confianza = 1 / (1 + np.exp(-np.max(decision_score))) 

    # Diccionario con el formato solicitado
    response = {
        "text": input_text,
        "prevision": str(label),
        "probabilidad": round(float(confianza), 2)
    }
    
    return response

## Ejemplo de uso
**Prueba Interactiva** </br>
Ejecuta la celda de abajo para ingresar una reseña manual. </br>
El resultado se mostrará formateado como un objeto JSON, tal como lo recibiría el cliente de la API.

In [4]:
# Captura de datos
user_input = input("Ingrese reseña para la API: ")

if user_input:
    resultado = predict_to_json(user_input)
    
    # Imprimimos con formato JSON bonito para documentación
    print("\n📦 RESPUESTA API (JSON):")
    print(json.dumps(resultado, indent=4, ensure_ascii=False))


📦 RESPUESTA API (JSON):
{
    "text": "feo",
    "prevision": "Negativo",
    "probabilidad": 0.48
}
